In [9]:
class singlePoint:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def get_x(self):
        return int(self.x)
    
    def get_y(self):
        return int(self.y)
    
    def get_position(self):
        return int(self.x),int(self.y)
    
class obstacle(singlePoint):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost = cost
        
    def get_cost(self):
        return int(self.cost)

class quickSand(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost=cost
        
#work on the range
class spiderNet(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost=cost
    
class fee(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost = cost 
        
class border(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost = cost

In [10]:
#read file and set up obstacles in arrays
import re
def readInput(file_name):
    input_file = open(file_name,"r")
    lines = input_file.readlines()

    line1 = lines[0]
    line2 = lines[1]
    line3 = lines[2]
    line4 = lines[3]
    line5 = lines[4]


    #CODE to get starting point in first line
    
    starting_location_coordinates = re.findall('\d+', line1)
    starting_coordinates = singlePoint(starting_location_coordinates[0],starting_location_coordinates[1])

    #CODE to get exit point in second line
    exit_location_coordinates = re.findall('\d+', line2)
    exit_coordinates = singlePoint(exit_location_coordinates[0],exit_location_coordinates[1])

    obstacles=[]

    #CODE to get quickSand point in third line
    quick_sands_location = re.findall('\d+', line3)
    number_of_quick_sands = 0

    for i in quick_sands_location[:]:
        number_of_quick_sands+=1

    if (number_of_quick_sands % 2 != 0):
        number_of_quick_sands = number_of_quick_sands - 1


    #quick_sand_array=[]
    for i in range(0,(int)(number_of_quick_sands/2)):
        quick_sand = quickSand(quick_sands_location[i*2],quick_sands_location[i*2+1],10000)
        obstacles.append(quick_sand)
    
    #CODE to get spider net point in fourth line
    spider_net_location = re.findall('\d+', line4)
    number_of_spider_nets=0

    for i in spider_net_location[:]:
        number_of_spider_nets+=1

    if ((number_of_spider_nets) % 2 != 0):
        number_of_spider_nets = number_of_spider_nets - 1

    initialize_spider_net_array=[]
    for i in range(0,int(number_of_spider_nets/2)):
        spider_net = spiderNet(spider_net_location[(i*2)],spider_net_location[(i*2+1)],0)
        initialize_spider_net_array.append(spider_net)

        
    #total_spider_net=[]        
    for spider in initialize_spider_net_array:
        x=int(spider.get_x())
        y=int(spider.get_y())
        for j in range(-1,2):
            #Avoid initializing spider web center twice
            if(j == 0):
                spider_net1 = spiderNet(x,y,10000)
                obstacles.append(spider_net1)
            else:
                spider_net1=spiderNet(x,y+j,10000)
                spider_net2=spiderNet(x+j,y,10000)
                obstacles.append(spider_net1)
                obstacles.append(spider_net2)
        
    #CODE to get fees point in fourth line
    fees_location_and_cost = re.findall('\d+', line5)
    number_of_fee_points = 0

    for i in fees_location_and_cost[:]:
        number_of_fee_points+=1

    fees_array=[]
    for i in range(0,(int)(number_of_fee_points/3)):
        fees = fee(fees_location_and_cost[i*3+1],fees_location_and_cost[i*3+2],fees_location_and_cost[i*3])
        obstacles.append(fees)
        
    return (starting_coordinates,exit_coordinates,obstacles)

In [11]:
class agent():
    current_x = 0
    current_y = 0
    def __init__ (self,starting_x,starting_y,exit_x,exit_y):
        self.starting_x=starting_x
        self.starting_y=starting_y
        self.exit_x=exit_x
        self.exit_y=exit_y
        
        #Current location
        self.current_x = starting_x
        self.current_y = starting_y
        
        #Calculates estimated cost to arrive at EXIT
        self.estimated_cost=abs(self.starting_x - self.exit_x)+abs(self.starting_y - self.exit_y)
        
        #Keeps track of cost
        self.steps_taken=0 
        
        self.cost_of_travel=0
        
    #Assumed h() function value returned  
    def distance_to_exit(self):
        distance = abs(self.current_x - self.exit_x)
        distance += abs(self.current_y - self.exit_y)
        return distance
        

    def move_up(self):
        self.current_y +=1
        self.steps_taken+=1
        self.cost_of_travel+=1
        
    def move_down(self):
        self.current_y -=1
        self.steps_taken+=1
        self.cost_of_travel+=1
        
    def move_right(self):
        self.current_x +=1
        self.steps_taken+=1
        self.cost_of_travel+=1
        
    def move_left(self):
        self.current_x -=1
        self.steps_taken+=1
        self.cost_of_travel+=1
        
    def current_position(self):
        return int(self.current_x),int(self.current_y)
    
    
    
    


In [12]:
#Reporting function
#Used when itfinds the EXIT
def report(d007, path):
    output_file = open("output.txt","w")
    
    seq = ["Total cost: ",str(d007.cost_of_travel) , '\n',
           "Total number of steps taken: \n",  str(d007.steps_taken),'\n'
           "Traveled route: " ]
    
    output_file.writelines(seq)

    for coord in path[:]:
        if (coord == path[-1]): 
            output_file.write(str(coord)) 
        else:
            output_file.write(str(coord) + ', ')
        
    output_file.close()


In [13]:
import random
def Astar(starting_coordinates,exit_coordinates,obstacles):
    de = [] #dead end list
    a=[]
    path=[]

    for i in range(0,4): #initialize array a
        a.append(0)
    double_o_seven = agent(starting_coordinates.get_x(),starting_coordinates.get_y(),exit_coordinates.get_x(),exit_coordinates.get_y())
    path.append(double_o_seven.current_position()) #add starting position to path

    while double_o_seven.distance_to_exit()>0:
   
        hueristic_values= []
        hueristic_values = h(double_o_seven.current_position(),exit_coordinates.get_position(),de,obstacles)
        #pass heuristic values
        a[0]=hueristic_values[0]
        a[1]=hueristic_values[1]
        a[2]= hueristic_values[2]
        a[3]=hueristic_values[3]
  
        minval = None #control logic : selects best heuristec value
        for index, val in enumerate(a):
            if minval is None or val < minval:
                indices = [index]
                minval = val
            elif val == minval:  #if multiple best values pick one randomly
                indices.append(index)
        choice = random.choice(indices)
        
        #navigation control for agent
        if(choice == 0):
            double_o_seven.move_up()
        elif(choice ==1):
            double_o_seven.move_right()
        elif(choice ==2):
            double_o_seven.move_down()
        elif(choice ==3):
            double_o_seven.move_left()
            
        for p in path: #check for visited
            if(double_o_seven.current_position() == p):
                de.append(path[-1])
                
                
        for obs in obstacles: #check if agent visited a fee
            if(double_o_seven.current_position() == obs.get_position()):
                double_o_seven.cost_of_travel+= obs.get_cost() 
        
        path.append(double_o_seven.current_position())

        
    #Print to file when EXIT found
    report(double_o_seven, path)
    return path


In [14]:
def h(coordinates,exit_coordinates,de,obstacles):
    heur=[]
    for i in range(0,4):
        heur.append(0)
        
    def look_up():
        temp_y = coordinates[1]
        temp_x = coordinates[0]
        exit_x = exit_coordinates[0]
        exit_y = exit_coordinates[1]
        cost=0 
        if (abs(temp_x - exit_x)+ abs(temp_y+1 - exit_y)== 0 ):  #close to exit
            return abs(temp_x - exit_x)+ abs(temp_y - exit_y) 
        elif temp_y == 19: #border
                cost+=1000000
                return cost 
        #look ahead
        else:
            temp_y +=1
            cost = abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            for i in de:
                if(temp_x == i[0] and temp_y== i[1]):
                    cost += 15  
            temp_y +=1
            cost += abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            return cost/2
    
    def look_down():
        temp_y = coordinates[1]
        temp_x = coordinates[0]
        exit_x = exit_coordinates[0]
        exit_y = exit_coordinates[1]
        cost=0
        if (abs(temp_x - exit_x)+ abs(temp_y-1 - exit_y)==0 ):
            return abs(temp_x - exit_x)+ abs(temp_y - exit_y)
        elif temp_y == 0:
            cost = 1000000
            return cost
        else: 
            temp_y -=1
            cost = abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            for i in de:
                if((temp_x,temp_y) == i):
                    cost += 15
            temp_y -=1
            cost += abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            return cost/2
        
    def look_right():
        temp_y = coordinates[1]
        temp_x = coordinates[0]
        exit_x = exit_coordinates[0]
        exit_y = exit_coordinates[1]
        cost=0 
        if (abs(temp_x+1 - exit_x)+ abs(temp_y - exit_y)== 0 ):
            return abs(temp_x - exit_x)+ abs(temp_y - exit_y)
        elif temp_x ==19:
            cost = 1000000
            return cost
        else:
            temp_x +=1
            cost = abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for d in de:
                if((temp_x,temp_y) == d):
                    cost += 15 
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            temp_x +=1
            cost += abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            return cost/2
        
        
    def look_left():
        temp_y = coordinates[1]
        temp_x = coordinates[0]
        exit_x = exit_coordinates[0]
        exit_y = exit_coordinates[1]
        cost=0
        if (abs(temp_x-1 - exit_x)+ abs(temp_y - exit_y)==0 ):
            return abs(temp_x - exit_x)+ abs(temp_y - exit_y)
        elif temp_x == 0:
            cost =1000000
            return cost
        else:
            temp_x -= 1
            cost = abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            for i in de:
                if(temp_x == i[0] and temp_y== i[1]):
                    cost += 15 
            temp_x -= 1
            cost += abs(temp_x - exit_x)+ abs(temp_y - exit_y)
            for obs in obstacles:
                if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                    cost += obs.get_cost()
            return cost/2
    
    heur[0]=look_up()
    heur[1]=look_right()
    heur[2]=look_down()
    heur[3]=look_left()
    return heur

In [15]:
import pylab as plot
def plotGraph(starting_coordinates,exit_coordinates,obstacles,path):
    plot.axis([0,20,0,20])
    
    for i in range(1,20):
        plot.plot([i,i],[0,20],'b-',linewidth=.2)
        plot.plot([0,20],[i,i],'b-',linewidth=.2)

    plot.plot(starting_coordinates.get_x(),starting_coordinates.get_y(),'go')
    plot.plot(exit_coordinates.get_x(),exit_coordinates.get_y(),'go')
    for ob in obstacles:
        plot.plot(ob.x,ob.y,"ro")
    path_length=len(path)
    for i in range(0,path_length):
        plot.plot(path[i][0], path[i][1],'ko')
    plot.show()


In [16]:
def maze():
        file_name="input.txt"
        starting_coordinates=readInput(file_name)[0]
        exit_coordinates= readInput(file_name)[1]
        obstacles =readInput(file_name)[2]
        path=Astar(starting_coordinates,exit_coordinates,obstacles)
        plotGraph(starting_coordinates,exit_coordinates,obstacles,path)

if __name__=="__main__":
    maze()
    